In [173]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import glob
import json
from pathlib import Path

In [174]:
def load_json(dir):
    try:
        with open(dir) as f:
            data = json.load(f)
            
        return data
    
    except:
        return None

In [175]:
RESULT_DIR = "..\\result_summary"

In [178]:
def parse_file_nm(file_nm):
    mcts_info = None
    
    components = file_nm.split("\\")
    # print(components)
    method = components[2]
    
    if 'am' in method:
        _, _, method, _, prob_type, num_probs, model_info, baseline_info, _ = components
        
    elif 'mcts' in method:
        _, _, method, _, prob_type, num_probs, model_info, baseline_info, mcts_info, _ = components


    
    num_probs = num_probs.split("-")[0]
    num_probs = int(num_probs.split("_")[1])
    baseline_info = baseline_info.split("-")[-1]
    
    return method, prob_type, num_probs, model_info, baseline_info, mcts_info

In [179]:
# parse all json file named with "all_result_avg.json" from the RESULT_DIR

all_result = {}

for file in glob.glob(f"{RESULT_DIR}/**/all_result_avg.json", recursive=True):
    data = load_json(file)
    # print(file)
    method, prob_type, num_probs, model_info, baseline_info, mcts_info = parse_file_nm(file)
    
    if data is not None:
        
        if method not in all_result:
            all_result[method] = {}
        
        if prob_type not in all_result[method]:
            all_result[method][prob_type] = {}
            
        if num_probs not in all_result[method][prob_type]:
            all_result[method][prob_type][num_probs] = {}
        
        if mcts_info is None:
            if model_info not in all_result[method][prob_type][num_probs]:
                all_result[method][prob_type][num_probs][model_info] = {}
            
            if baseline_info not in all_result[method][prob_type][num_probs][model_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info] = data
                
        else:
            if model_info not in all_result[method][prob_type][num_probs]:
                all_result[method][prob_type][num_probs][model_info] = {}
                
            if baseline_info not in all_result[method][prob_type][num_probs][model_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info] = {}
                
            if mcts_info not in all_result[method][prob_type][num_probs][model_info][baseline_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info] = data 
                
    # break

In [180]:
def get_parital_result(method, prob_type, num_prob):       
    partial_result = {'model_info': [], 'baseline': [], 'mcts_info': [], 'epoch': [], 'score': [], 'runtime': [], 'score_std': []}
    
    def _append_epoch_result(model_info, baseline_info, epoch, mcts, epoch_score, epoch_score_std, epoch_runtime):
        partial_result['model_info'].append(model_info)
        partial_result['baseline'].append(baseline_info)
        partial_result['mcts_info'].append(mcts)
        partial_result['epoch'].append(epoch)                
        partial_result['score'].append(epoch_score)
        partial_result['score_std'].append(epoch_score_std)
        partial_result['runtime'].append(epoch_runtime)
        

    target_result = all_result[method][prob_type][num_prob]
    
    if "am" in method:
        for model_info in target_result:
            for baseline_info in target_result[model_info]:
                all_epoch_results = target_result[model_info][baseline_info]
                
                for epoch in all_epoch_results:
                    current_epoch_result = all_epoch_results[epoch]['result_avg']
                    epoch_score = current_epoch_result['score']
                    epoch_score_std = all_epoch_results[epoch]['result_std']['score']
                    epoch_runtime = current_epoch_result['runtime']
                    
                    _append_epoch_result(model_info, baseline_info, epoch, "am", epoch_score,epoch_score_std, epoch_runtime)
                    
    elif 'mcts' in method:
        for model_info in target_result:
            for baseline_info in target_result[model_info]:
                all_mcts_results = target_result[model_info][baseline_info]
                
                for mcts_info in all_mcts_results:
                    all_epoch_results = all_mcts_results[mcts_info]
                    
                    for epoch in all_epoch_results:
                        current_epoch_result = all_epoch_results[epoch]['result_avg']
                        epoch_score = current_epoch_result['score']
                        epoch_score_std = all_epoch_results[epoch]['result_std']['score']
                        epoch_runtime = current_epoch_result['runtime']
                        
                        _append_epoch_result(model_info, baseline_info, epoch, mcts_info, epoch_score,epoch_score_std, epoch_runtime)                                         
            
    df = pd.DataFrame.from_dict(partial_result)
    df['activation'] = df['model_info'].apply(lambda x: x.split("-")[5])
    df = df.drop(columns=['model_info'])
    
    df['train_score'] = df.apply(lambda x: x['epoch'].split("-")[1].split("=")[1], axis=1).astype(float)
    df['epoch'] =  df.apply(lambda x: x['epoch'].split("-")[0].split("=")[1], axis=1).astype(int)
    
    if 'mcts' in method:
        df['cpuct'] = df['mcts_info'].apply(lambda x: x.split("-")[2].split('_')[1])
        df['ns'] = df['mcts_info'].apply(lambda x: x.split("-")[0].split('_')[1])
        df['cpuct'] = df['cpuct'].astype(float)
        df['ns'] = df['ns'].astype(int)
    
    elif 'am' in method:
        df['cpuct'] = 0
        df['ns'] = 0
        
    df = df.drop(columns=['mcts_info'])
    

        
    return df

In [181]:
def plot_bar_result(base_df, baseline, activation, prob_type, num_prob, plot_dev=False, hue='cpuct'):
    if prob_type == 'tsp':
        y_ranges = {20: (3.75, 3.95), 50: (5.7, 5.875), 100: (7.95, 8.25)}
        
    elif prob_type == 'cvrp':
        y_ranges = {20: (6.1, 7.5), 50: (9, 12), 100: (7.95, 8.25)}
        
    _df = base_df[(base_df['baseline'] == baseline) & (base_df['activation'] == activation)]
    
    # drop rows where the mcts_info is 0.8
    _df = _df[_df['cpuct'] != '0.8']
    
    # sort by the train score
    _df = _df.sort_values(by=['epoch', 'cpuct', 'ns']).reset_index(drop=True)
    _df['score_std'] = _df['score_std'].astype(float)
    
    _df['cpuct'] = _df['cpuct'].astype(object)
    _df['cpuct'] = _df['cpuct'].fillna('am')
    
    _df['ns'] = _df['ns'].astype(object)
    _df['ns'] = _df['ns'].fillna('am')
    
    plt.figure(figsize=(12, 7))
    
    ax = sns.barplot(data=_df, x=_df['epoch'], y='score', hue=hue)
    
    if plot_dev:
        bar_coords = [(rect.get_x(), rect.get_y(), rect.get_width(), rect.get_height()) for rect in ax.patches]
                
        for i, (x, y, w, h) in enumerate(bar_coords):
            x_pos = x + w / 2
            y_top = y + h + 0.01
            
            plt.errorbar(x=x_pos, y=y_top, yerr=_df['score_std'][i], fmt='none', color='black', capsize=4)
    
    plt.title(f"{prob_type}_{num_prob}_{baseline}_{activation}")
    plt.ylim(*y_ranges[num_prob])
    plt.legend()
    
    path = Path(f"../result_image/bars")
    
    if not path.exists():
        path.mkdir(parents=True)
        
    
    plt.savefig(f"{path}/{prob_type}_{num_prob}_{baseline}_{activation}.png")
    plt.show()

In [182]:
# tsp_20 = pd.concat([get_parital_result('am', 'tsp', 20), get_parital_result('mcts_v2', 'tsp', 20)])

# for _baseline in ['mean', 'val']:
#     for _activation in ['relu', 'swiglu']:
#         plot_bar_result(tsp_20, _baseline, _activation, 'tsp', 20, hue='cpuct')

In [183]:
tsp_100 = pd.concat([get_parital_result('am', 'tsp', 100), get_parital_result('mcts_v2', 'tsp', 100)])
tsp_100.sort_values(by=['score'])

,baseline,epoch,score,runtime,score_std,activation,train_score,cpuct,ns
21,val,191,8.451261,67.104227,0.149279,swiglu,7.80099,1.1,250
3,val,191,8.491095,0.138628,0.159627,swiglu,7.80099,0.0,0
2,mean,181,8.506774,306.072845,0.223587,relu,7.88802,1.1,2000
0,mean,181,8.529293,0.130679,0.342256,relu,7.88802,0.0,0
18,val,191,8.535767,28.175558,0.218532,swiglu,7.80099,1.1,100
23,val,191,8.564041,123.831510,0.459309,swiglu,7.80099,1.1,500
4,mean,181,8.573574,883.778280,0.371488,relu,7.88802,1.1,4000
5,mean,181,8.621124,133.504730,0.382694,relu,7.88802,1.1,500
1,mean,181,8.685923,244.597380,0.710522,relu,7.88802,1.1,1000
19,val,191,8.699914,235.063877,0.353413,swiglu,7.80099,1.1,1000


In [184]:
tsp_100.pivot_table(index=['epoch', 'ns', 'activation', 'baseline'], values=['score', 'r'], aggfunc=[np.mean, np.std])

mean           
                                    runtime      score
epoch ns   activation baseline                        
121   0    relu       val          0.131202   9.041163
      100  relu       val         29.407412   9.332102
      250  relu       val         78.312759   8.967759
      500  relu       val        155.438322   9.095035
      1000 relu       val        294.773021   9.276087
      2000 relu       val        374.322330   9.526844
      4000 relu       val       1139.096735   9.774798
169   0    swiglu     mean         0.130285   9.087556
      100  swiglu     mean        30.698916   9.682075
      250  swiglu     mean        76.176870   9.693900
      500  swiglu     mean       150.919923  10.368583
      1000 swiglu     mean       307.894682  12.067280
      2000 swiglu     mean       397.565899  13.052202
      4000 swiglu     mean      1234.981797  14.223460
181   0    relu       mean         0.130679   8.529293
      100  relu       mean        29.268141   8.711763
      250  relu       mean        74.997932   8.798154
      500  relu       mean       133.504730   8.621124
      1000 relu       mean       244.597380   8.685923
      2000 relu       mean       306.072845   8.506774
      4000 relu       mean       883.778280   8.573574
191   0    swiglu     val          0.138628   8.491095
      100  swiglu     val         28.175558   8.535767
      250  swiglu     val         67.104227   8.451261
      500  swiglu     val        123.831510   8.564041
      1000 swiglu     val        235.063877   8.699914
      2000 swiglu     val        288.398647   8.939841
      4000 swiglu     val        888.426925   8.890822

In [162]:
# for _baseline in ['mean', 'val']:
#     for _activation in ['relu', 'swiglu']:
#         plot_bar_result(tsp_100, _baseline, _activation, 'tsp', 100, hue='cpuct')


In [50]:
# tsp_50 = pd.concat([get_parital_result('am', 'tsp', 50), get_parital_result('mcts', 'tsp', 50, leave_only_puct=True)])

# for _baseline in ['mean', 'val']:
#     for _activation in ['relu', 'swiglu']:
#         plot_bar_result(tsp_50, _baseline, _activation, 'tsp', 50)

In [51]:
# tsp_100 = pd.concat([get_parital_result('am', 'tsp', 100), get_parital_result('mcts', 'tsp', 100, leave_only_puct=True)])

# for _baseline in ['mean', 'val']:
#     for _activation in ['relu', 'swiglu']:
#         plot_bar_result(tsp_100, _baseline, _activation, 'tsp', 100)